# Peephole LSTM Test & Performance Comparison (Speed & Memory)

* [Imports](#Importing-necessary-modules)
* [Load & Definition](#Loading-and-defining-modules)
    * [Autograd Functions](#Autograd-Functions)
    * [Module Classes](#Module-classes-(C++,-CUDA,-PyTorch))
* [Models](#Defining-models)
    * [Definition](#Definition)
    * [Instantiation](#Instantiation)
    * [Parameter Synchronization](#Parameter-Synchronization)
* [Fake Dataset](#Creating-a-fake-dataset)
* [Sanity Check](#Sanity-check:-output-comparison)
    * [Forward Outputs](#Forward-Outputs)
    * [Backward Gradients](#Backward-Gradients)
* [Forward Performance](#Forward-time-comparison)
* [+Backward Performance](#+Backward-time-comparison)

---

## Importing necessary modules
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [40]:
if 'initialized' not in globals():
    import torch
    from torch import nn
    from torch.utils.cpp_extension import load
    from torch.nn import functional as F
    from torch.utils.data import TensorDataset, DataLoader

    import math
    from collections import OrderedDict
    from time import sleep

    initialized = [False] * 7
    print(torch.__version__)

---

## Loading and defining modules
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

### Autograd Functions
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [41]:
if not initialized[0]:
    _ln_peephole_lstm_layer_cpp = load('ln_peephole_lstm_layer', ['./ln_peephole_lstm_layer.cpp'])
    _ln_peephole_lstm_layer_cuda = load('ln_peephole_lstm_layer_cuda', ['./ln_peephole_lstm_layer_cuda.cpp', './ln_peephole_lstm_layer_cuda_kernel.cu'])

    ########################################################################################################################

    class LNPeepholeLSTMFunctionCPP(torch.autograd.Function):
        @staticmethod
        def forward(ctx, input, weight_ih, weight_hh, weight_ch, bias,
                    gamma_f, gamma_i, gamma_g, gamma_o, gamma_cell, beta_cell,
                    hidden, cell,
                    epsilon, dropout_p,
                    dropout_output, training):

            outputs = _ln_peephole_lstm_layer_cpp.forward(input, weight_ih, weight_hh, weight_ch, bias,
                                                          gamma_f, gamma_i, gamma_g, gamma_o, gamma_cell, beta_cell,
                                                          hidden, cell,
                                                          epsilon, dropout_p,
                                                          dropout_output, training)

            out, new_h, new_cell = outputs[:3]

            variables = outputs[3:] + [weight_ih, weight_hh, weight_ch,
                                       gamma_f, gamma_i, gamma_g, gamma_o, gamma_cell]
            ctx.save_for_backward(*variables)

            return out, new_h, new_cell

        @staticmethod
        def backward(ctx, grad_output, grad_h, grad_cell):
            outputs = _ln_peephole_lstm_layer_cpp.backward(
                grad_output.contiguous(), grad_h.contiguous(), grad_cell.contiguous(), *ctx.saved_tensors)

            (d_input, d_weight_ih, d_weight_hh, d_weight_ch, d_bias,
             d_gamma_f, d_gamma_i, d_gamma_o, d_gamma_g, d_gamma_cell, d_beta_cell,
             d_hidden, d_cell) = outputs

            return (d_input, d_weight_ih, d_weight_hh, d_weight_ch, d_bias,
                    d_gamma_f, d_gamma_i, d_gamma_o, d_gamma_g, d_gamma_cell, d_beta_cell,
                    d_hidden, d_cell,
                    None, None,
                    None, None)
        
    ########################################################################################################################
    
    class LNPeepholeLSTMFunctionCUDA(torch.autograd.Function):
        @staticmethod
        def forward(ctx, input, weight_ih, weight_hh, weight_ch, bias,
                    gamma_f, gamma_i, gamma_g, gamma_o, gamma_cell, beta_cell,
                    hidden, cell,
                    epsilon, dropout_p,
                    dropout_output, training):

            outputs = _ln_peephole_lstm_layer_cuda.forward(input, weight_ih, weight_hh, weight_ch, bias,
                                                          gamma_f, gamma_i, gamma_g, gamma_o, gamma_cell, beta_cell,
                                                          hidden, cell,
                                                          epsilon, dropout_p,
                                                          dropout_output, training)

            out, new_h, new_cell = outputs[:3]

            variables = outputs[3:] + [weight_ih, weight_hh, weight_ch,
                                       gamma_f, gamma_i, gamma_g, gamma_o, gamma_cell]
            ctx.save_for_backward(*variables)

            return out, new_h, new_cell

        @staticmethod
        def backward(ctx, grad_output, grad_h, grad_cell):
            outputs = _ln_peephole_lstm_layer_cuda.backward(
                grad_output.contiguous(), grad_h.contiguous(), grad_cell.contiguous(), *ctx.saved_tensors)

            (d_input, d_weight_ih, d_weight_hh, d_weight_ch, d_bias,
             d_gamma_f, d_gamma_i, d_gamma_o, d_gamma_g, d_gamma_cell, d_beta_cell,
             d_hidden, d_cell) = outputs

            return (d_input, d_weight_ih, d_weight_hh, d_weight_ch, d_bias,
                    d_gamma_f, d_gamma_i, d_gamma_o, d_gamma_g, d_gamma_cell, d_beta_cell,
                    d_hidden, d_cell,
                    None, None,
                    None, None)
        
    initialized[0] = True

### Module classes (PyTorch, C++, CUDA)
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [42]:
if not initialized[1]:
    class LNPeepholeLSTMTorch(nn.Module):
        def __init__(self, input_size, hidden_size, batch_first=False, dropout=0., dropout_on_output=True, eps=1e-05):
            if not 0 <= dropout <= 1:
                raise ValueError(f"Invalid dropout value : {dropout} dropout must be in range [0, 1].")
            super(LNPeepholeLSTMTorch, self).__init__()

            self.input_size = input_size
            self.hidden_size = hidden_size
            self.batch_first = bool(batch_first)
            self.dropout = float(dropout)
            self.dropout_on_output = bool(dropout_on_output)
            self.eps = eps

            self.register_parameter('weight_ih', nn.Parameter(torch.empty(4 * hidden_size, input_size)))
            self.register_parameter('weight_hh', nn.Parameter(torch.empty(4 * hidden_size, hidden_size)))
            self.register_parameter('weight_ch', nn.Parameter(torch.empty(3 * hidden_size)))
            self.register_parameter('bias', nn.Parameter(torch.empty(4 * hidden_size)))

            self.register_parameter('gamma_f', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_i', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_g', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_o', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_cell', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('beta_cell', nn.Parameter(torch.empty(hidden_size)))

            self.reset_parameters()

        def reset_parameters(self):
            stdv = 1.0 / math.sqrt(self.input_size + 2 * self.hidden_size)
            self.weight_ih.data.uniform_(-stdv, +stdv)
            self.weight_hh.data.uniform_(-stdv, +stdv)
            self.weight_ch.data.uniform_(-stdv, +stdv)

            self.bias.data.zero_()
            self.bias.data[:self.hidden_size].fill_(1.)

            self.gamma_f.data.uniform_()
            self.gamma_i.data.uniform_()
            self.gamma_g.data.uniform_()
            self.gamma_o.data.uniform_()
            self.gamma_cell.data.uniform_()
            self.beta_cell.data.zero_()

        def forward(self, input, states):
            assert input.dim() == 3, "expected a 3 dimensional tensor as `input`, but te given tensor has {} dimension(s)".format(input.dim())
            assert len(states) == 2, "expected a (hidden, cell) pair as `states`, but the length of the given states is {}".format(len(states))
            if self.batch_first:
                input = input.transpose(0, 1).contiguous()
            assert states[0].size() == (input.size(1), self.hidden_size), "expected a hidden state tensor with dimensionality {}, but the given tensor has dimensionality []".format(list(states[0].size()), [input.size(1), self.hidden_size])
            assert states[1].size() == (input.size(1), self.hidden_size), "expected a cell state tensor with dimensionality {}, but the given tensor has dimensionality []".format(list(states[1].size()), [input.size(1), self.hidden_size])

            hidden, cell = states

            hidden_size = self.hidden_size
            hidden_size_2 = 2 * hidden_size
            hidden_size_3 = hidden_size_2 + hidden_size

            norm_shape = torch.Size((hidden_size,))

            outputs = input.new_empty((input.size(0), input.size(1), hidden_size))
            
            ih = input.matmul(self.weight_ih.t())

            weight_hc_h = torch.cat((self.weight_hh.t(),
                                     torch.cat((self.weight_ch[:hidden_size].diag(),
                                                self.weight_ch[hidden_size:hidden_size_2].diag(),
                                                self.weight_ch.new_zeros(hidden_size_2, hidden_size))).t()))
            weight_co = self.weight_ch[hidden_size_2:]
            
            gamma_fig = torch.stack((self.gamma_f, self.gamma_i, self.gamma_g))

            bias_fig = torch.stack(self.bias[:hidden_size_3].chunk(3, dim=0))
            bias_o = self.bias[hidden_size_3:]

            for i in range(input.size(0)):
                gates = torch.addmm(ih[i], torch.cat((hidden, cell), dim=1), weight_hc_h).view(-1, 4, hidden_size)
                gates_fig = gates[:, :3]


                gates_fig = F.layer_norm(gates_fig, norm_shape, eps=self.eps)
                gates_fig = torch.addcmul(bias_fig, gates_fig, gamma_fig)
                forget_input_gates = gates_fig[:, :2].sigmoid()
                candidate_cell = F.dropout(gates_fig[:, 2].tanh(), p=self.dropout, training=self.training)


                cell = F.layer_norm(torch.addcmul(forget_input_gates[:, 0] * cell,
                                                  forget_input_gates[:, 1], candidate_cell),
                                    norm_shape, self.gamma_cell, self.beta_cell, self.eps)

                output_gate = torch.addcmul(gates[:, 3], cell, weight_co)

                output_gate = F.layer_norm(output_gate, norm_shape, self.gamma_o, bias_o, self.eps).sigmoid()

                hidden = output_gate * cell.tanh()

                outputs[i] = hidden

            if self.dropout_on_output:
                outputs = F.dropout(outputs, p=self.dropout, training=self.training)
                
            if self.batch_first:
                outputs = outputs.transpose(0, 1).contiguous()

            return outputs, (hidden, cell)

        def __repr__(self):
            return f"LNPeepholeLSTMTorch(input_size={self.input_size}, hidden_size={self.hidden_size}, batch_first={self.batch_first}, dropout={self.dropout}, dropout_on_output={self.dropout_on_output}, eps={self.eps})"

    ########################################################################################################################

    class LNPeepholeLSTMCPP(nn.Module):
        def __init__(self, input_size, hidden_size, batch_first=False, dropout=0., dropout_on_output=True, eps=1e-05):
            if not 0 <= dropout <= 1:
                raise ValueError(f"Invalid dropout value : {dropout} dropout must be in range [0, 1].")

            super(LNPeepholeLSTMCPP, self).__init__()

            self.input_size = input_size
            self.hidden_size = hidden_size
            self.batch_first = bool(batch_first)
            self.dropout = float(dropout)
            self.dropout_on_output = bool(dropout_on_output)
            self.eps = eps

            self.register_parameter('weight_ih', nn.Parameter(torch.empty(4 * hidden_size, input_size)))
            self.register_parameter('weight_hh', nn.Parameter(torch.empty(4 * hidden_size, hidden_size)))
            self.register_parameter('weight_ch', nn.Parameter(torch.empty(3 * hidden_size)))
            self.register_parameter('bias', nn.Parameter(torch.empty(4 * hidden_size)))

            self.register_parameter('gamma_f', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_i', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_g', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_o', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_cell', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('beta_cell', nn.Parameter(torch.empty(hidden_size)))

            self.reset_parameters()

        def reset_parameters(self):
            stdv = 1.0 / math.sqrt(self.input_size + 2 * self.hidden_size)
            self.weight_ih.data.uniform_(-stdv, +stdv)
            self.weight_hh.data.uniform_(-stdv, +stdv)
            self.weight_ch.data.uniform_(-stdv, +stdv)

            self.bias.data.zero_()
            self.bias.data[:self.hidden_size].fill_(1.)

            self.gamma_f.data.uniform_()
            self.gamma_i.data.uniform_()
            self.gamma_g.data.uniform_()
            self.gamma_o.data.uniform_()
            self.gamma_cell.data.uniform_()
            self.beta_cell.data.zero_()

        def forward(self, input, state):
            if self.batch_first:
                input = input.transpose(0, 1).contiguous()

            output, new_h, new_cell = LNPeepholeLSTMFunctionCPP.apply(
                input, self.weight_ih, self.weight_hh, self.weight_ch, self.bias,
                self.gamma_f, self.gamma_i, self.gamma_g, self.gamma_o, self.gamma_cell, self.beta_cell,
                state[0], state[1],
                self.eps, self.dropout, self.dropout_on_output, self.training)
            
            if self.batch_first:
                output = output.transpose(0, 1).contiguous()

            return output, (new_h, new_cell)

        def __repr__(self):
            return f"LNPeepholeLSTMCPP(input_size={self.input_size}, hidden_size={self.hidden_size}, batch_first={self.batch_first}, dropout={self.dropout}, dropout_on_output={self.dropout_on_output}, eps={self.eps})"
    
    ########################################################################################################################
    
    class LNPeepholeLSTMCUDA(nn.Module):
        def __init__(self, input_size, hidden_size, batch_first=False, dropout=0., dropout_on_output=True, eps=1e-05):
            if not 0 <= dropout <= 1:
                raise ValueError(f"Invalid dropout value : {dropout} dropout must be in range [0, 1].")

            super(LNPeepholeLSTMCUDA, self).__init__()

            self.input_size = input_size
            self.hidden_size = hidden_size
            self.batch_first = bool(batch_first)
            self.dropout = float(dropout)
            self.dropout_on_output = bool(dropout_on_output)
            self.eps = eps

            self.register_parameter('weight_ih', nn.Parameter(torch.empty(4 * hidden_size, input_size)))
            self.register_parameter('weight_hh', nn.Parameter(torch.empty(4 * hidden_size, hidden_size)))
            self.register_parameter('weight_ch', nn.Parameter(torch.empty(3 * hidden_size)))
            self.register_parameter('bias', nn.Parameter(torch.empty(4 * hidden_size)))

            self.register_parameter('gamma_f', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_i', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_g', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_o', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('gamma_cell', nn.Parameter(torch.empty(hidden_size)))
            self.register_parameter('beta_cell', nn.Parameter(torch.empty(hidden_size)))

            self.reset_parameters()

        def reset_parameters(self):
            stdv = 1.0 / math.sqrt(self.input_size + 2 * self.hidden_size)
            self.weight_ih.data.uniform_(-stdv, +stdv)
            self.weight_hh.data.uniform_(-stdv, +stdv)
            self.weight_ch.data.uniform_(-stdv, +stdv)

            self.bias.data.zero_()
            self.bias.data[:self.hidden_size].fill_(1.)

            self.gamma_f.data.uniform_()
            self.gamma_i.data.uniform_()
            self.gamma_g.data.uniform_()
            self.gamma_o.data.uniform_()
            self.gamma_cell.data.uniform_()
            self.beta_cell.data.zero_()

        def forward(self, input, state):
            if self.batch_first:
                input = input.transpose(0, 1).contiguous()

            output, new_h, new_cell = LNPeepholeLSTMFunctionCUDA.apply(
                input, self.weight_ih, self.weight_hh, self.weight_ch, self.bias,
                self.gamma_f, self.gamma_i, self.gamma_g, self.gamma_o, self.gamma_cell, self.beta_cell,
                state[0], state[1],
                self.eps, self.dropout, self.dropout_on_output, self.training)
            
            if self.batch_first:
                output = output.transpose(0, 1).contiguous()

            return output, (new_h, new_cell)

        def __repr__(self):
            return f"LNPeepholeLSTMCUDA(input_size={self.input_size}, hidden_size={self.hidden_size}, batch_first={self.batch_first}, dropout={self.dropout}, dropout_on_output={self.dropout_on_output}, eps={self.eps})"
    
    initialized[1] = True

---

## Defining models
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

### Definition
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [43]:
if not initialized[2]:
    class LNPeepholeTorch(nn.Module):
        def __init__(self, input_size, hidden_size, output_size, n_layers=1, dropout=0, dropout_on_output=True, eps=1e-05):
            super().__init__()
            assert isinstance(n_layers, int)
            assert n_layers > 0

            self.lstm0 = LNPeepholeLSTMTorch(input_size, hidden_size, True, dropout, dropout_on_output, eps)
            for n in range(1, n_layers):
                self.add_module("lstm{}".format(n), LNPeepholeLSTMTorch(hidden_size, hidden_size, True, dropout, dropout_on_output, eps))
            self.fc = nn.Linear(hidden_size, output_size)

            self.input_size = input_size
            self.hidden_size = hidden_size
            self.output_size = output_size
            self.n_layers = n_layers

        def forward(self, x, states):
            assert states[0].dim() == 3
            assert states[0].size(0) == self.n_layers
            assert states[0].size(1) == x.size(0)
            assert states[0].size(2) == self.hidden_size
            assert states[0].size() == states[1].size()

            new_hidden = torch.empty_like(states[0])
            new_cell = torch.empty_like(states[1])

            for n in range(self.n_layers):
                x, (new_hidden[n], new_cell[n]) = getattr(self, "lstm{}".format(n))(x, (states[0][n], states[1][n]))
            x = self.fc(x)

            return x, (new_hidden, new_cell)

    ########################################################################################################################

    class LNPeepholeCPP(nn.Module):
        def __init__(self, input_size, hidden_size, output_size, n_layers=1, dropout=0, dropout_on_output=True, eps=1e-05):
            super().__init__()
            assert isinstance(n_layers, int)
            assert n_layers > 0

            self.lstm0 = LNPeepholeLSTMCPP(input_size, hidden_size, True, dropout, dropout_on_output, eps)
            for n in range(1, n_layers):
                self.add_module("lstm{}".format(n), LNPeepholeLSTMCPP(hidden_size, hidden_size, True, dropout, dropout_on_output, eps))
            self.fc = nn.Linear(hidden_size, output_size)

            self.input_size = input_size
            self.hidden_size = hidden_size
            self.output_size = output_size
            self.n_layers = n_layers

        def forward(self, x, states):
            assert states[0].dim() == 3
            assert states[0].size(0) == self.n_layers
            assert states[0].size(1) == x.size(0)
            assert states[0].size(2) == self.hidden_size
            assert states[0].size() == states[1].size()

            new_hidden = torch.empty_like(states[0])
            new_cell = torch.empty_like(states[1])

            for n in range(self.n_layers):
                x, (new_hidden[n], new_cell[n]) = getattr(self, "lstm{}".format(n))(x, (states[0][n], states[1][n]))
            x = self.fc(x)

            return x, (new_hidden, new_cell)
    
    ########################################################################################################################
    
    class LNPeepholeCUDA(nn.Module):
        def __init__(self, input_size, hidden_size, output_size, n_layers=1, dropout=0, dropout_on_output=True, eps=1e-05):
            super().__init__()
            assert isinstance(n_layers, int)
            assert n_layers > 0

            self.lstm0 = LNPeepholeLSTMCUDA(input_size, hidden_size, True, dropout, dropout_on_output, eps)
            for n in range(1, n_layers):
                self.add_module("lstm{}".format(n), LNPeepholeLSTMCUDA(hidden_size, hidden_size, True, dropout, dropout_on_output, eps))
            self.fc = nn.Linear(hidden_size, output_size)

            self.input_size = input_size
            self.hidden_size = hidden_size
            self.output_size = output_size
            self.n_layers = n_layers

        def forward(self, x, states):
            assert states[0].dim() == 3
            assert states[0].size(0) == self.n_layers
            assert states[0].size(1) == x.size(0)
            assert states[0].size(2) == self.hidden_size
            assert states[0].size() == states[1].size()

            new_hidden = torch.empty_like(states[0])
            new_cell = torch.empty_like(states[1])

            for n in range(self.n_layers):
                x, (new_hidden[n], new_cell[n]) = getattr(self, "lstm{}".format(n))(x, (states[0][n], states[1][n]))
            x = self.fc(x)

            return x, (new_hidden, new_cell)
    
    initialized[2] = True

### Instantiation
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [44]:
if not initialized[3]:
    device = ('cpu', 'cuda')[1]

    input_size = 5 #TEST 5
    hidden_size = 8 #TEST 8
    output_size = 6 #TEST 6
    n_layers = 3 #TEST 3
    dropout = 0. #TEST 0
    eps = 1e-05 #TEST 1e-05

    model_torch = LNPeepholeTorch(input_size, hidden_size, output_size, n_layers, dropout, eps)
    model_cpp = LNPeepholeCPP(input_size, hidden_size, output_size, n_layers, dropout, eps)
    model_cuda = LNPeepholeCUDA(input_size, hidden_size, output_size, n_layers, dropout, eps)

    model_torch.to(device)
    model_cpp.to(device)
    model_cuda.to(device)

    models = (model_torch, model_cpp, model_cuda)
    
    initialized[3] = True

### Parameter Synchronization
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [45]:
if not initialized[4]:
    named_parameter_dicts = [
        dict(model_torch.named_parameters()),
        dict(model_cpp.named_parameters()),
        dict(model_cuda.named_parameters())
    ]

    print("Synchronized Parameters:\n")
    for common_param_name in set.intersection(*(set(npd.keys()) for npd in named_parameter_dicts)):
        print("\t{}".format(common_param_name))
        for i in range(1, len(named_parameter_dicts)):
            if named_parameter_dicts[i][common_param_name].size() == named_parameter_dicts[0][common_param_name].size():
                named_parameter_dicts[i][common_param_name].data = named_parameter_dicts[0][common_param_name].data
            else:
                raise RuntimeError("Size mismatch\n0:{}\n{i}:{}".format(named_parameter_dicts[0][common_param_name].size(),
                                                                        named_parameter_dicts[i][common_param_name].size()))
    print()
    print("Exclusive Parameters (Not Synchronized):\n")
    for exclusive_param_name in set.union(*(set(npd.keys()) for npd in named_parameter_dicts)) - set.intersection(*(set(npd.keys()) for npd in named_parameter_dicts)):
        print("\t{}".format(exclusive_param_name))
        
    initialized[4] = True

---

## Creating a fake dataset
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [46]:
if not initialized[5]:
    def create_fake_loader(dataset_size, sequence_length, batch_size, drop_last=True):
        fake_inputs = torch.randn(dataset_size, sequence_length, input_size)
        fake_targets = torch.randint(high=output_size, size=(dataset_size, sequence_length), dtype=torch.int64)

        fake_dataset = TensorDataset(fake_inputs, fake_targets)

        fake_loader = DataLoader(fake_dataset, batch_size=batch_size, drop_last=drop_last)

        return fake_loader
    
    initialized[5] = True

In [47]:
if not initialized[6]:
    dataset_size = 1000
    sequence_length = 20 #TEST 20
    batch_size = 8 #TEST 8

    fake_loader = create_fake_loader(dataset_size, sequence_length, batch_size)
    print(next(iter(fake_loader))[0].size(), next(iter(fake_loader))[1].size())
    
    initialized[6] = True

---

## Sanity check: output comparison
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

### Forward Outputs
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [11]:
hidden = (torch.zeros(n_layers, batch_size, hidden_size, device=device), torch.zeros(n_layers, batch_size, hidden_size, device=device))

inputs = next(iter(fake_loader))[0].to(device)

for model in models:
    model.train()
#     model.eval()
del model # Removing temporary variable

In [12]:
with torch.no_grad():
    print("[model_torch]")
    print("\n{partial output}")
    out0 = model_torch(inputs, hidden)
    print(out0)

[model_torch]

{partial output}
(tensor([[[ 0.1133,  0.0837, -0.2642,  0.0978, -0.0272, -0.3072],
         [ 0.0137,  0.0775, -0.2885,  0.0676,  0.0583, -0.2692],
         [-0.0021,  0.0668, -0.2956,  0.0365,  0.0884, -0.2731],
         [ 0.0035,  0.0485, -0.2546, -0.0066,  0.1256, -0.3038],
         [ 0.0223,  0.0816, -0.2180, -0.0449,  0.1738, -0.3202],
         [ 0.0797,  0.0964, -0.2087, -0.0854,  0.1761, -0.3625],
         [ 0.0838,  0.1359, -0.2150, -0.1281,  0.2035, -0.3626],
         [ 0.0439,  0.1890, -0.2321, -0.1536,  0.2456, -0.3128],
         [ 0.0334,  0.2178, -0.2362, -0.1545,  0.2487, -0.2813],
         [-0.0012,  0.2589, -0.2308, -0.1648,  0.2831, -0.2359],
         [-0.0075,  0.2794, -0.2265, -0.1569,  0.2887, -0.2114],
         [-0.0137,  0.2942, -0.2215, -0.1552,  0.2949, -0.1930],
         [-0.0157,  0.3032, -0.2184, -0.1527,  0.2964, -0.1813],
         [-0.0170,  0.3091, -0.2170, -0.1528,  0.2971, -0.1729],
         [-0.0148,  0.3098, -0.2173, -0.1502,  0.2945, -0

In [13]:
with torch.no_grad():
    print("[model_cpp]")
    print("\n{partial output}")
    out1 = model_cpp(inputs, hidden)
    print(out1)

[model_cpp]

{partial output}
(tensor([[[ 0.1133,  0.0837, -0.2642,  0.0978, -0.0272, -0.3072],
         [ 0.0137,  0.0775, -0.2885,  0.0676,  0.0583, -0.2692],
         [-0.0021,  0.0668, -0.2956,  0.0365,  0.0884, -0.2731],
         [ 0.0035,  0.0485, -0.2546, -0.0066,  0.1256, -0.3038],
         [ 0.0223,  0.0816, -0.2180, -0.0449,  0.1738, -0.3202],
         [ 0.0797,  0.0964, -0.2087, -0.0854,  0.1761, -0.3625],
         [ 0.0838,  0.1359, -0.2150, -0.1281,  0.2035, -0.3626],
         [ 0.0439,  0.1890, -0.2321, -0.1536,  0.2456, -0.3128],
         [ 0.0334,  0.2178, -0.2362, -0.1545,  0.2487, -0.2813],
         [-0.0012,  0.2589, -0.2308, -0.1648,  0.2831, -0.2359],
         [-0.0075,  0.2794, -0.2265, -0.1569,  0.2887, -0.2114],
         [-0.0137,  0.2942, -0.2215, -0.1552,  0.2949, -0.1930],
         [-0.0157,  0.3032, -0.2184, -0.1527,  0.2964, -0.1813],
         [-0.0170,  0.3091, -0.2170, -0.1528,  0.2971, -0.1729],
         [-0.0148,  0.3098, -0.2173, -0.1502,  0.2945, -0.1

In [14]:
with torch.no_grad():
    print(out0[0].sub(out1[0]).abs().sum())

tensor(6.4198e-05, device='cuda:0')


In [15]:
with torch.no_grad():
    print("[model_cuda]")
    print("\n{partial output}")
    out2 = model_cuda(inputs, hidden)
    print(out2)

[model_cuda]

{partial output}
(tensor([[[ 0.1133,  0.0837, -0.2642,  0.0978, -0.0272, -0.3072],
         [ 0.0137,  0.0775, -0.2885,  0.0676,  0.0583, -0.2692],
         [-0.0021,  0.0668, -0.2956,  0.0365,  0.0884, -0.2731],
         [ 0.0035,  0.0485, -0.2546, -0.0066,  0.1256, -0.3038],
         [ 0.0223,  0.0816, -0.2180, -0.0449,  0.1738, -0.3202],
         [ 0.0797,  0.0964, -0.2087, -0.0854,  0.1761, -0.3625],
         [ 0.0838,  0.1359, -0.2150, -0.1281,  0.2035, -0.3626],
         [ 0.0439,  0.1890, -0.2321, -0.1536,  0.2456, -0.3128],
         [ 0.0334,  0.2178, -0.2362, -0.1545,  0.2487, -0.2813],
         [-0.0012,  0.2589, -0.2308, -0.1648,  0.2831, -0.2359],
         [-0.0075,  0.2794, -0.2265, -0.1569,  0.2887, -0.2114],
         [-0.0137,  0.2942, -0.2215, -0.1552,  0.2949, -0.1930],
         [-0.0157,  0.3032, -0.2184, -0.1527,  0.2964, -0.1813],
         [-0.0170,  0.3091, -0.2170, -0.1528,  0.2971, -0.1729],
         [-0.0148,  0.3098, -0.2173, -0.1502,  0.2945, -0.

In [16]:
with torch.no_grad():
    print(out0[0].sub(out2[0]).abs().sum())

tensor(3.0879e-05, device='cuda:0')


### Backward Gradients
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [25]:
criterion = nn.CrossEntropyLoss()

hidden = (torch.zeros(n_layers, batch_size, hidden_size, device=device, requires_grad=True), torch.zeros(n_layers, batch_size, hidden_size, device=device, requires_grad=True))

inputs, targets = next(iter(fake_loader))
inputs = inputs.to(device)
inputs.requires_grad_()
targets = targets.to(device)

inputs_grads = []
hidden_grads = []
for model in models:
    model.train()
#     model.eval()
    model.zero_grad()
    loss = criterion(model(inputs, hidden)[0].flatten(0, 1), targets.flatten(0, 1))
    print(loss)
    loss.backward()
    inputs_grads.append(inputs.grad.clone())
    inputs.grad.zero_()
    hidden_grads.append((hidden[0].grad.clone(), hidden[1].grad.clone()))
    hidden[0].grad.zero_()
    hidden[1].grad.zero_()

tensor(1.8411, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.8411, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.8411, device='cuda:0', grad_fn=<NllLossBackward>)


In [26]:
print("model_torch")
print(inputs_grads[0])
print(hidden_grads[0])
print(model_torch.lstm0.bias.grad)
print(model_torch.lstm0.gamma_cell.grad)
# print(model_torch.lstm0.weight_ih.grad[0 * hidden_size:1 * hidden_size])
# print(model_torch.lstm0.weight_ih.grad[1 * hidden_size:2 * hidden_size])
# print(model_torch.lstm0.weight_ih.grad[2 * hidden_size:3 * hidden_size])
# print(model_torch.lstm0.weight_ih.grad[3 * hidden_size:4 * hidden_size])

model_torch
tensor([[[-1.3509e-03, -1.6698e-03,  7.7635e-04, -1.5118e-03, -1.1353e-03],
         [-4.5543e-05, -9.8839e-04,  8.8485e-05, -1.5346e-03, -6.6258e-04],
         [-2.2454e-04, -7.8383e-04,  5.4592e-05, -1.2401e-03,  8.5120e-06],
         [-1.6555e-04,  3.0245e-05,  2.6124e-04, -3.3674e-04, -2.2809e-04],
         [-2.9974e-05,  1.1698e-04,  3.0914e-05, -1.7709e-05,  5.3651e-05],
         [-1.3969e-05,  2.3306e-04,  3.1881e-04, -1.7451e-04, -1.0234e-04],
         [-2.1236e-05,  3.8609e-05,  8.6061e-05, -3.2555e-04, -2.0546e-04],
         [-1.0516e-04, -1.2785e-04,  2.7324e-05, -2.4500e-04, -2.1783e-04],
         [-1.2939e-04, -1.6066e-04, -6.3855e-06, -4.0414e-05, -9.0024e-05],
         [-5.1257e-05, -6.3875e-05,  2.4613e-05, -8.1985e-05, -5.8957e-05],
         [-4.5431e-05, -8.5340e-05,  2.1641e-05, -1.3855e-04, -9.9256e-05],
         [-2.3020e-05, -5.9431e-05,  2.2959e-05, -8.3060e-05, -5.3722e-05],
         [-2.9868e-05, -6.0172e-05,  1.3853e-05, -4.2396e-05, -1.7778e-05],


           2.8282e-05,  3.4200e-04, -1.8732e-04]]], device='cuda:0'))
tensor([ 4.4736e-03, -3.8939e-03, -1.8645e-03, -3.7163e-03, -2.0731e-03,
        -6.5457e-05,  1.1893e-06,  3.5572e-03, -1.5572e-03, -1.5776e-03,
        -1.0119e-02,  2.8923e-03,  7.1374e-04,  1.1776e-04,  1.3337e-03,
         7.9236e-03, -6.7504e-03,  1.7586e-01,  1.9664e-02,  1.3147e-03,
         2.9830e-02, -1.5023e-02, -1.4171e-02, -1.2627e-01,  1.4383e-03,
        -2.6038e-03, -4.5237e-03, -1.5763e-03,  1.8966e-03,  1.8134e-03,
        -9.0131e-05,  1.2650e-02], device='cuda:0')
tensor([ 0.0401, -0.0259, -0.0139, -0.0223,  0.0058,  0.0103, -0.0358,  0.0573],
       device='cuda:0')


In [27]:
print("model_cpp")
print(inputs_grads[1])
print(hidden_grads[1])
print(model_cpp.lstm0.bias.grad)
print(model_cpp.lstm0.gamma_cell.grad)
# print(model_cpp.lstm0.weight_ih.grad[0 * hidden_size:1 * hidden_size])
# print(model_cpp.lstm0.weight_ih.grad[1 * hidden_size:2 * hidden_size])
# print(model_cpp.lstm0.weight_ih.grad[2 * hidden_size:3 * hidden_size])
# print(model_cpp.lstm0.weight_ih.grad[3 * hidden_size:4 * hidden_size])

model_cpp
tensor([[[-1.3509e-03, -1.6698e-03,  7.7635e-04, -1.5118e-03, -1.1353e-03],
         [-4.5542e-05, -9.8839e-04,  8.8485e-05, -1.5346e-03, -6.6258e-04],
         [-2.2454e-04, -7.8383e-04,  5.4592e-05, -1.2401e-03,  8.5126e-06],
         [-1.6555e-04,  3.0245e-05,  2.6124e-04, -3.3674e-04, -2.2809e-04],
         [-2.9974e-05,  1.1698e-04,  3.0914e-05, -1.7709e-05,  5.3651e-05],
         [-1.3969e-05,  2.3306e-04,  3.1881e-04, -1.7451e-04, -1.0234e-04],
         [-2.1236e-05,  3.8609e-05,  8.6061e-05, -3.2555e-04, -2.0546e-04],
         [-1.0516e-04, -1.2785e-04,  2.7324e-05, -2.4500e-04, -2.1783e-04],
         [-1.2939e-04, -1.6066e-04, -6.3855e-06, -4.0414e-05, -9.0024e-05],
         [-5.1257e-05, -6.3875e-05,  2.4613e-05, -8.1985e-05, -5.8957e-05],
         [-4.5431e-05, -8.5340e-05,  2.1641e-05, -1.3855e-04, -9.9256e-05],
         [-2.3020e-05, -5.9431e-05,  2.2959e-05, -8.3060e-05, -5.3722e-05],
         [-2.9868e-05, -6.0172e-05,  1.3853e-05, -4.2396e-05, -1.7778e-05],
  

(tensor([[[-2.5189e-04, -2.0082e-04, -1.7558e-04, -8.0521e-04,  1.1709e-04,
           2.7003e-05, -4.6220e-04, -9.6273e-04],
         [-2.5566e-02,  1.4535e-02,  2.0867e-02, -1.8974e-02,  9.1213e-03,
           2.2444e-02, -1.2072e-03, -1.9303e-02],
         [ 1.8213e-03,  5.0340e-03, -2.8973e-03, -2.5948e-03, -5.9670e-03,
          -1.1212e-02,  4.1239e-03, -2.4724e-03],
         [-5.1882e-04, -9.1419e-04,  1.6955e-03,  8.0996e-04,  8.9677e-04,
           3.9048e-03,  1.3760e-04,  1.4093e-03],
         [ 1.9941e-03,  4.7367e-05, -1.0460e-03,  2.0620e-03, -2.5649e-04,
          -3.3732e-03,  4.8564e-04,  1.4308e-03],
         [-1.7544e-04,  4.4309e-04, -1.4960e-03, -1.8501e-03, -6.7367e-04,
          -5.5798e-04, -1.1859e-03, -2.0914e-03],
         [-1.1434e-02,  1.1806e-02,  3.5609e-03, -4.9980e-03,  1.7843e-03,
          -1.2426e-02,  5.8125e-04, -5.9034e-03],
         [ 5.2873e-03, -5.2022e-03, -4.5778e-03,  1.9382e-03, -1.7460e-03,
          -3.0236e-04, -6.7515e-04,  1.4303e-03]]

In [28]:
print(model_torch.lstm0.weight_ih.grad.sub(model_cpp.lstm0.weight_ih.grad).abs().sum())

tensor(6.2985e-05, device='cuda:0')


In [29]:
print("model_cuda")
print(inputs_grads[2])
print(hidden_grads[2])
print(model_cuda.lstm0.bias.grad)
print(model_cuda.lstm0.gamma_cell.grad)
# print(model_cuda.lstm0.weight_ih.grad[0 * hidden_size:1 * hidden_size])
# print(model_cuda.lstm0.weight_ih.grad[1 * hidden_size:2 * hidden_size])
# print(model_cuda.lstm0.weight_ih.grad[2 * hidden_size:3 * hidden_size])
# print(model_cuda.lstm0.weight_ih.grad[3 * hidden_size:4 * hidden_size])

model_cuda
tensor([[[-1.3509e-03, -1.6698e-03,  7.7635e-04, -1.5118e-03, -1.1353e-03],
         [-4.5542e-05, -9.8839e-04,  8.8485e-05, -1.5346e-03, -6.6258e-04],
         [-2.2454e-04, -7.8383e-04,  5.4592e-05, -1.2401e-03,  8.5128e-06],
         [-1.6555e-04,  3.0245e-05,  2.6124e-04, -3.3674e-04, -2.2809e-04],
         [-2.9974e-05,  1.1698e-04,  3.0914e-05, -1.7709e-05,  5.3651e-05],
         [-1.3969e-05,  2.3306e-04,  3.1881e-04, -1.7451e-04, -1.0234e-04],
         [-2.1236e-05,  3.8609e-05,  8.6061e-05, -3.2555e-04, -2.0546e-04],
         [-1.0516e-04, -1.2785e-04,  2.7324e-05, -2.4500e-04, -2.1783e-04],
         [-1.2939e-04, -1.6066e-04, -6.3855e-06, -4.0414e-05, -9.0024e-05],
         [-5.1257e-05, -6.3875e-05,  2.4613e-05, -8.1985e-05, -5.8957e-05],
         [-4.5431e-05, -8.5340e-05,  2.1641e-05, -1.3855e-04, -9.9256e-05],
         [-2.3020e-05, -5.9431e-05,  2.2959e-05, -8.3060e-05, -5.3722e-05],
         [-2.9868e-05, -6.0172e-05,  1.3853e-05, -4.2396e-05, -1.7778e-05],
 

           2.8282e-05,  3.4200e-04, -1.8732e-04]]], device='cuda:0'))
tensor([ 4.4736e-03, -3.8939e-03, -1.8645e-03, -3.7163e-03, -2.0731e-03,
        -6.5458e-05,  1.1893e-06,  3.5571e-03, -1.5572e-03, -1.5776e-03,
        -1.0119e-02,  2.8923e-03,  7.1374e-04,  1.1776e-04,  1.3337e-03,
         7.9236e-03, -6.7505e-03,  1.7586e-01,  1.9664e-02,  1.3147e-03,
         2.9830e-02, -1.5023e-02, -1.4171e-02, -1.2627e-01,  1.4383e-03,
        -2.6038e-03, -4.5237e-03, -1.5763e-03,  1.8965e-03,  1.8134e-03,
        -9.0131e-05,  1.2650e-02], device='cuda:0')
tensor([ 0.0401, -0.0259, -0.0139, -0.0223,  0.0058,  0.0103, -0.0358,  0.0573],
       device='cuda:0')


In [30]:
print(model_torch.lstm0.weight_ih.grad.sub(model_cuda.lstm0.weight_ih.grad).abs().sum())

tensor(5.3484e-06, device='cuda:0')


---

## Forward time comparison
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [39]:
dataset_size = 1000
sequence_length = 20
batch_size = 32

for model in models:
    model.train()
del model

fake_loader = create_fake_loader(dataset_size, sequence_length, batch_size, drop_last=True)

hidden = (torch.zeros(n_layers, batch_size, hidden_size, device=device), torch.zeros(n_layers, batch_size, hidden_size, device=device))

In [48]:
%%timeit -n 1 -r 20
with torch.no_grad():
    for inputs, _ in fake_loader:
        inputs = inputs.to(device)
        model_torch(inputs, hidden)

3.6 s ± 155 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)


In [57]:
%%timeit -n 1 -r 20
with torch.no_grad():
    for inputs, _ in fake_loader:
        inputs = inputs.to(device)
        model_cpp(inputs, hidden)

5.26 s ± 87.1 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)


In [58]:
%%timeit -n 1 -r 20
with torch.no_grad():
    for inputs, _ in fake_loader:
        inputs = inputs.to(device)
        model_cuda(inputs, hidden)

1.26 s ± 60.1 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)


---

## +Backward time comparison
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [48]:
criterion = nn.CrossEntropyLoss()
"executed"

'executed'

In [76]:
%%timeit -r 20
for inputs, targets in fake_loader:
    inputs = inputs.to(device)
    targets = targets.to(device)
    model_torch.zero_grad()
    criterion(model_torch(inputs, hidden)[0].flatten(0, 1), targets.flatten(0, 1)).backward()

14 s ± 234 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)


In [85]:
%%timeit -r 20
for inputs, targets in fake_loader:
    inputs = inputs.to(device)
    targets = targets.to(device)
    model_cpp.zero_grad()
    criterion(model_cpp(inputs, hidden)[0].flatten(0, 1), targets.flatten(0, 1)).backward()

10.1 s ± 173 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)


In [49]:
%%timeit -r 20
for inputs, targets in fake_loader:
    inputs = inputs.to(device)
    targets = targets.to(device)
    model_cuda.zero_grad()
    criterion(model_cuda(inputs, hidden)[0].flatten(0, 1), targets.flatten(0, 1)).backward()

2.41 s ± 42.6 ms per loop (mean ± std. dev. of 20 runs, 1 loop each)


---